Import libraries

In [125]:
# Import all libraries regarding torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde, lognorm
import os
from torch.utils.data import Dataset, DataLoader
import time
import shutil

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.datasets as datasets

Read Input data 

In [126]:
# Reading from RSA_input.csv
df = pd.read_csv('RSA_input.csv') # add csv file to the correct location
grain_R = df["grain_R"]
grain_asp = df["grain_asp"]
print(grain_R.max())
print(grain_R.min())
print(grain_asp.max())
print(grain_asp.min())
print(len(grain_R)) # length 30218
print(len(grain_asp)) # length 30218

# Combined into numpy shape (30218, 2)
grainsData = np.column_stack((grain_R, grain_asp))
print(grainsData.shape)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

108.8169461
3.989422804
6.209501941
1.0
30218
30218
(30218, 2)


Create NN models

In [127]:
# Establish convention for real and fake labels during training
real_label = 1.
fake_label = -1.

# Generator
class Generator(nn.Module):
    def __init__(self, latent_Gaussian_dimension, number_of_grain_features):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # change width and depth of the network here
            nn.Linear(latent_Gaussian_dimension, 120),
            #nn.BatchNorm1d(120),
            nn.ReLU(True),

            nn.Linear(120, 80),
            #nn.BatchNorm1d(80),
            nn.ReLU(True),
            
            nn.Linear(80, 40),
            #nn.BatchNorm1d(40),
            nn.ReLU(True),
            
            nn.Linear(40, number_of_grain_features),
            #nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, number_of_grain_features):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # change width and depth of the network here
            nn.Linear(number_of_grain_features, 40),
            #nn.BatchNorm1d(40),
            nn.ReLU(),
            
            nn.Linear(40, 80),
            #nn.BatchNorm1d(80),
            nn.ReLU(),
            
            nn.Linear(80, 120),
            #nn.BatchNorm1d(120),
            nn.ReLU(),
            
            nn.Linear(120, 160),
            #nn.BatchNorm1d(160),
            nn.ReLU(),

            nn.Linear(160, 1)

            # The output of discriminator is no longer a probability, 
            # we do not apply sigmoid at the output of discriminator.
        )

    def forward(self, x):
        return self.model(x)

Train the Models

In [128]:
# Hyperparameters
learning_rate = 5e-5
batch_size = 3000
generator_iterations = 2
critic_iterations = 3
weight_clipping_limit = 1
num_epochs = 1000
printAndSaveEvery_N_Epoch = 1
# Initialize generator and discriminator
latent_Gaussian_dimension = 160  # Dimension of the input noise vector
number_of_grain_features = 2  # Dimension of the real data
#real_data_dim = 30218  # Dimension of the real data
number_of_reduced_grains = 1000  # Dimension of the generated data

In [129]:
# initialize gen and discriminator_loss
generator = Generator(latent_Gaussian_dimension, number_of_grain_features).to(device)
discriminator = Discriminator(number_of_grain_features).to(device)

#initialize optimizer
generator_optimizer = optim.RMSprop(generator.parameters(), lr=learning_rate)
discriminator_optimizer = optim.RMSprop(discriminator.parameters(), lr=learning_rate)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create DataLoader
dataloader = DataLoader(grainsData, batch_size=batch_size, shuffle=True)

# initialize tensorboard plotting

# train gen & critic in loop
for epoch in range(0, num_epochs + 2):
    g_loss_list = []
    #d_loss_list = []
    d_loss_real_list = []
    d_loss_fake_list = []
    Wasserstein_distance_list = []
    
    for batch_index, real_grains in enumerate(dataloader):
        real_grains = real_grains.to(torch.float32)
        # Requires grad, Generator requires_grad = False

        one = torch.FloatTensor([1])
        minus_one = one * -1
            
        for gen_iter in range(generator_iterations): 
            for param in discriminator.parameters():
                param.requires_grad = True
            # Train discriminator = max E(discriminator(real) - E(dicriminator(fake)))
            #d_loss_iter = []
            d_loss_real_iter = []
            d_loss_fake_iter = []
            Wasserstein_distance_iter = []
            for critic_iter in range(critic_iterations):
                discriminator.zero_grad()

                # Clamp parameters to a range [-c, c], c=self.weight_cliping_limit
                #for param in discriminator.parameters():
                #    param.data.clamp_(-weight_clipping_limit, weight_clipping_limit)

                # Train discriminator
                # WGAN - Training discriminator more iterations than generator
                # Train with real grains

                d_loss_real = discriminator(real_grains)
                d_loss_real = d_loss_real.mean().view(1)
                d_loss_real.backward(minus_one)

                # Train with fake grains
                noise = torch.randn(batch_size, latent_Gaussian_dimension, device=device, dtype=torch.float32)
                fake_grains = generator(noise)

                d_loss_fake = discriminator(fake_grains)
                d_loss_fake = d_loss_fake.mean().view(1)
                d_loss_fake.backward(one)

                d_loss = d_loss_fake - d_loss_real
                Wasserstein_distance = d_loss_real - d_loss_fake
                discriminator_optimizer.step()

                #d_loss_iter.append(d_loss.item())
                d_loss_real_iter.append(d_loss_real.item())
                d_loss_fake_iter.append(d_loss.item())
                Wasserstein_distance_iter.append(Wasserstein_distance.item())
            
            #d_loss_list.append(np.mean(d_loss_iter))
            d_loss_real_list.append(np.mean(d_loss_real_iter))
            d_loss_fake_list.append(np.mean(d_loss_fake_iter))
            Wasserstein_distance_list.append(np.mean(Wasserstein_distance_iter))
            # print(f'Discriminator iteration: {critic_iter}/{critic_iterations}, loss_fake: {d_loss_fake.data}, loss_real: {d_loss_real.data}')

            # Generator update
            for param in discriminator.parameters():
                param.requires_grad = False  # to avoid computation

            generator.zero_grad()

            # Train generator
            # Compute loss with fake grains
            noise = torch.randn(batch_size, latent_Gaussian_dimension, device=device, dtype=torch.float32)
            fake_grains = generator(noise)
            g_loss = discriminator(fake_grains)
            g_loss = g_loss.mean().view(1)
            g_loss.backward(minus_one)
            generator_optimizer.step()

            g_loss_list.append(g_loss.item())
            # print(f'Generator iteration: {gen_iter}/{generator_iterations}, g_loss: {g_loss.data}')
    
    ############################
    # Print training progress
    ###########################
    
    if epoch % printAndSaveEvery_N_Epoch == 0:
        g_loss_value = np.mean(g_loss_list)
        d_loss_real_value = np.mean(d_loss_real_list)
        d_loss_fake_value = np.mean(d_loss_fake_list)
        #d_loss = np.mean(d_loss_list)
        Wasserstein_distance_value = np.mean(Wasserstein_distance_list)
        print(f"Epoch {epoch}")
        print(f"Generator Loss: {g_loss_value:.4f}")
        print(f"Discriminator Loss: d_loss_real={d_loss_real_value:.4f}, d_loss_fake={d_loss_fake_value:.4f}")
        print(f"Wasserstein Distance: {Wasserstein_distance_value:.4f}")
        
    
    if epoch % printAndSaveEvery_N_Epoch == 0:
        with torch.no_grad():
            # Generate fake grains and save into csv file
            generate_times = 20
            # for each epoch, we wil generate 20 different groups of fake grains as generator is random
            outputPath = f"downsampled_grains/epoch_{epoch}"
            if os.path.exists(outputPath):
                # remove the directory if it exists and create new
                shutil.rmtree(outputPath)
                os.makedirs(outputPath)
            else:
                os.makedirs(outputPath)
            
            for i in range(generate_times):  
                outputPathIndex = f"downsampled_grains/epoch_{epoch}" 
                noise = torch.randn(number_of_reduced_grains, latent_Gaussian_dimension, device=device)
                fake_grains = generator(noise)
                fake_grains = fake_grains.cpu().numpy()
                #print(fake_grains.shape)
                # Save as csv file, use pandas
                columns = ["grain_R", "grain_asp"]
                df = pd.DataFrame(fake_grains, columns=columns)
                df.to_csv(f"{outputPathIndex}/grains_{i+1}.csv", index=False)

                

Epoch 0
Generator Loss: -0.0252
Discriminator Loss: d_loss_real=3.0107, d_loss_fake=-3.0337
Wasserstein Distance: 3.0337
Epoch 1
Generator Loss: -0.0669
Discriminator Loss: d_loss_real=10.1945, d_loss_fake=-10.2579
Wasserstein Distance: 10.2579
Epoch 2
Generator Loss: 0.0055
Discriminator Loss: d_loss_real=22.5139, d_loss_fake=-22.5040
Wasserstein Distance: 22.5040
Epoch 3
Generator Loss: 0.6205
Discriminator Loss: d_loss_real=41.6614, d_loss_fake=-41.0451
Wasserstein Distance: 41.0451
Epoch 4
Generator Loss: 3.2250
Discriminator Loss: d_loss_real=68.6034, d_loss_fake=-65.4109
Wasserstein Distance: 65.4109
Epoch 5
Generator Loss: 9.8685
Discriminator Loss: d_loss_real=105.3289, d_loss_fake=-95.5773
Wasserstein Distance: 95.5773
Epoch 6
Generator Loss: 22.9706
Discriminator Loss: d_loss_real=154.4157, d_loss_fake=-131.6459
Wasserstein Distance: 131.6459
Epoch 7
Generator Loss: 46.0919
Discriminator Loss: d_loss_real=215.5263, d_loss_fake=-169.8832
Wasserstein Distance: 169.8832
Epoch 8


KeyboardInterrupt: 